In [ ]:
import numpy as np
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def leaky_relu(x):
    if x >=0:
        return x
    else:
        return -0.01*x
    
def d_leaky_relu(x):
    if x >=0:
        return 1.0
    else:
        return -0.01
    
def softmax(x):
    x -= np.max(x, axis=0)
    return (np.exp(x))/(np.sum(np.exp(x), axis=0).T)

def grad_clip(x):
    #if x >=0.01:
    #    return 0.01
    #elif x<=-0.01:
    #    return -0.01
    #else:
    #    return x
    return x
    
def loss(pred, y):
    loss = -1.*np.sum(y*np.log(pred+0.001)/y.shape[1])
    return loss

def fw_prop(X, Y, W, b):
    m = X.shape[2]
    n_0 = X.shape[0]*X.shape[1]
    n_1 = W[1].shape[0]
    n_2 = W[2].shape[0]
    n_3 = W[3].shape[0]
    A_0 = X.reshape((n_0,m))
    
    Z_1 = np.dot(W[1], A_0.reshape((n_0,m))) + b[1]
    A_1 = np.vectorize(leaky_relu)(Z_1)
    
    Z_2 = np.dot(W[2], A_1.reshape((n_1,m))) + b[2]
    A_2 = np.vectorize(leaky_relu)(Z_2)
    
    Z_3 = np.dot(W[3], A_2.reshape((n_2,m))) + b[3]
    A_3 = softmax(Z_3)
    
    score = loss(A_3, Y)
    return A_3, J

def fw_bk_prop(X, Y, W, b, alpha):
    m = X.shape[2]
    n_0 = X.shape[0]*X.shape[1]
    n_1 = W[1].shape[0]
    n_2 = W[2].shape[0]
    n_3 = W[3].shape[0]
    epsilon = 10e-5
    A_0 = X.reshape((n_0,m))
    
    Z_1 = np.dot(W[1], A_0.reshape((n_0,m))) + b[1]
    A_1 = np.vectorize(leaky_relu)(Z_1)
    dA_1 = np.vectorize(d_leaky_relu)(Z_1)
    Z_1_papprox = np.dot(W[1], A_0.reshape((n_0,m))) + b[1]
    A_1_papprox = np.vectorize(leaky_relu)(Z_1_papprox)
    Z_1_mapprox = np.dot(W[1], A_0.reshape((n_0,m))) + b[1]
    A_1_mapprox = np.vectorize(leaky_relu)(Z_1_mapprox)
    
    Z_2 = np.dot(W[2], A_1.reshape((n_1,m))) + b[2]
    A_2 = np.vectorize(leaky_relu)(Z_2)
    dA_2 = np.vectorize(d_leaky_relu)(Z_2)
    Z_2_papprox = np.dot(W[2], A_1_papprox.reshape((n_1,m))) + b[1]
    A_2_papprox = np.vectorize(leaky_relu)(Z_2_papprox)
    Z_2_mapprox = np.dot(W[2], A_1_mapprox.reshape((n_1,m))) + b[1]
    A_2_mapprox = np.vectorize(leaky_relu)(Z_2_mapprox)
    
    Z_3 = np.dot(W[3], A_2.reshape((n_2,m))) + b[3]
    A_3 = softmax(Z_3)
    Z_3_papprox = np.dot(W[3]+epsilon, A_2_papprox.reshape((n_2,m))) + b[3]
    A_3_papprox = softmax(Z_3_papprox)
    Z_3_mapprox = np.dot(W[3]-epsilon, A_2_mapprox.reshape((n_2,m))) + b[3]
    A_3_mapprox = softmax(Z_3_mapprox)
    
    score = loss(A_3, Y)
    papprox = loss(A_3_papprox, Y)
    mapprox = loss(A_3_mapprox, Y)
    d_approx = (papprox-mapprox)/(2*epsilon)
    
    dZ_3 = A_3 - Y
    dW_3 = (1./m)*np.dot(dZ_3,A_2.T)
    db_3 = (1./m)*np.sum(dZ_3, axis=1, keepdims=True)
    
    dZ_2 = np.dot(W[3].T,dZ_3)*dA_2
    dW_2 = (1./m)*np.dot(dZ_2,A_1.T)
    db_2 = (1./m)*np.sum(dZ_2, axis=1, keepdims=True)
    
    dZ_1 = np.dot(W[2].T,dZ_2)*dA_1
    dW_1 = (1./m)*np.dot(dZ_1,A_0.T)
    db_1 = (1./m)*np.sum(dZ_1, axis=1, keepdims=True)
    
    #print(A_3 - Y)
    #print("Z_3: " + str(np.max(np.absolute(Z_3))))
    #print(np.linalg.norm(d_approx-dW_3))
    #print(np.linalg.norm(d_approx))
    #print(np.linalg.norm(dW_3))
    #print(np.linalg.norm(d_approx-dW_3)/(np.linalg.norm(d_approx)+np.linalg.norm(dW_3)))
    #print("dW_1: " + str(np.max(np.absolute(dW_1))))
    #print("dW_2: " + str(np.max(np.absolute(dW_2))))
    #print("dW_3: " + str(np.max(np.absolute(dW_3))))
    #print("db_1: " + str(np.max(np.absolute(db_1))))
    #print("db_2: " + str(np.max(np.absolute(db_2))))
    #print("db_3: " + str(np.max(np.absolute(db_3))))
    W[1]-=alpha*np.vectorize(grad_clip)(dW_1)
    W[2]-=alpha*np.vectorize(grad_clip)(dW_2)
    W[3]-=alpha*np.vectorize(grad_clip)(dW_3)
    b[1]-=alpha*np.vectorize(grad_clip)(db_1)
    b[2]-=alpha*np.vectorize(grad_clip)(db_2)
    b[3]-=alpha*np.vectorize(grad_clip)(db_3)
    
    return W, b, score

In [ ]:
digits = load_digits()
X = digits.images
mu = np.mean(X, axis=0)
sigma = np.var(X, axis=0)
X = (X-mu)/(sigma+10e-8)
labels = digits.target
Y = np.zeros((X.shape[0], 10))

for i in range(0, labels.shape[0]):
    Y[i][labels[i]] = 1

    
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)
X_train = X_train.T
X_test = X_test.T
Y_train = Y_train.T
Y_test = Y_test.T
n_sample_train = X_train.shape[2]
mini_batch = 64
number_mini_batch = n_sample_train/mini_batch + 1


In [ ]:
W = []
b = []

#W_1 = (1./64)*(np.random.sample((128,64)))
#b_1 = np.zeros((128,1))
#
#W_2 = (1./128)*(np.random.sample((128,128)))
#b_2 = np.zeros((128,1))
#
#W_3 = (1./128)*(np.random.sample((10,128)))
#b_3 = np.zeros((10,1))

W_1 = np.load('save/w1.npz')['arr_0']
W_2 = np.load('save/w2.npz')['arr_0']
W_3 = np.load('save/w3.npz')['arr_0']
b_1 = np.load('save/b1.npz')['arr_0']
b_2 = np.load('save/b2.npz')['arr_0']
b_3 = np.load('save/b3.npz')['arr_0']

W.append(0)
W.append(W_1)
W.append(W_2)
W.append(W_3)
b.append(0)
b.append(b_1)
b.append(b_2)
b.append(b_3)

W_2.shape

In [ ]:
number_epoch = 1000
train_losses = []
test_losses = []
for e in range(0, number_epoch):
    for i in range(0, number_mini_batch):
        if i!=number_mini_batch-1:
            W, b, J = fw_bk_prop(X_train[:,:,mini_batch*i:mini_batch*(i+1)], Y_train[:,mini_batch*i:mini_batch*(i+1)], W, b, alpha=0.01)
            #print(np.max(b[1]))
            #print(np.max(b[2]))
            #print(np.max(b[3]))
        else:
            W, b, J = fw_bk_prop(X_train[:,:,mini_batch*i:], Y_train[:,mini_batch*i:], W, b, alpha=0.0001/(e+1001))
            
    train_losses.append(J)
    pred, test_score = fw_prop(X_test, Y_test, W, b)
    test_losses.append(test_score)
    if e%10 == 0:
        print("J: " + str(J))

In [ ]:
mu = np.mean(X, axis=0)
sigma = np.var(X, axis=0)

In [ ]:
X_test_orig = X_test.T*sigma + mu
plt.imshow((X_test.T*sigma + mu)[359])

In [ ]:
for j in range(0, 10):
    x = X_test[:,:,j].reshape((8,8,1))
    y = Y_test[:,j].reshape((10,1))
    preds, score = fw_prop(x, y, W, b)
    plt.imshow(X_test_orig[j])
    print(np.argmax(preds))

In [ ]:
np.savez_compressed('e2000_w1.npz',W[1])
np.savez_compressed('e2000_w2.npz',W[2])
np.savez_compressed('e2000_w3.npz',W[3])
np.savez_compressed('e2000_b1.npz',b[1])
np.savez_compressed('e2000_b2.npz',b[2])
np.savez_compressed('e2000_b3.npz',b[3])



In [ ]:
X_test[:,:,2]